# Multiple reaction equilibrium: Gibbs energy minimization approach

## Example 1:
$I+B\rightleftharpoons P$

This example involves no matrix math.


In [1]:
######## Gibbs energy minimization approach for solving chemical equilibrium - one reaction

### Import packages

import numpy as np
from scipy.optimize import minimize

### Species vector
A = ['I','B','P1','P2']

### Stoichometric matrix, single reaction
nu = np.array([-1,-1,1,0])

### Reaction thermodynamics
K = 108

delta = nu.sum()   # Calculate delta by summing along rows
n_species = len(nu)    # Number of species = number of columns in nu

### Initial conditions
y0 = np.array([0.5,0.5,0,0])
P = 2.5  #atm
Pref = 1 #atm

### Guess reaction extent
xip_g = 0.4

###
def Gmin(U): 
    xip = U[0]
    
    y = (y0 + nu*xip)/(1+delta*xip) # mole fraction
    a = P/Pref*y # activity
    
    ### First term
    G_term1 = -xip*np.log(K)      

    ### Second term
    G_term2 = 0 
    for j in range(n_species):
        if a[j]>0:
            G_term2 = G_term2 + (y0[j]+nu[j]*xip)*np.log(a[j])
        else:
            G_term2 = G_term2

    Gm = G_term1 + G_term2
    
    return Gm

def ineq1(U):
    xip = U
    y = (y0 + nu*xip)/(1+delta*xip)
    greaterthanzero = y.T[0].tolist()
    
    return greaterthanzero
    
### Run minimization
soln = minimize(Gmin,xip_g,method='trust-constr',constraints=dict(type='ineq',fun=ineq1))

### Report whether solver successful and solution
print('Success =',soln.success,'\n')

print('Gmin = \n',soln.fun,'\n')

xipsoln = soln.x[0] 
print('xip = \n',xipsoln,'\n')

ysoln = (y0 + nu*xipsoln)/(1+delta*xipsoln)
print('y = \n',ysoln)

Success = True 

Gmin = 
 -1.9437408050401674 

xip = 
 0.4696241576750836 

y = 
 [0.0572723  0.0572723  0.88545541 0.        ]


## Example 2:
$I+B\rightleftharpoons P1$

$I+B\rightleftharpoons P2$

This example DOES involve matrix math

In [2]:
######## Gibbs energy minimization approach for solving chemical equilibrium

### Import packages

import numpy as np
from scipy.optimize import minimize

### Species vector
A = ['I','B','P1','P2']

### Stoichometric matrix
nu = np.array([[-1,-1,1,0],
               [-1,-1,0,1]])

### Reaction thermodynamics
K = np.array([[108,284]]).T

### Choose reactions
rxns = [0] # Specify a list (e.g., [0] will pick only reaction 1, [0,1] will pick the first and second reaction)

nu=nu[rxns,:]
K = K[rxns,:]

delta = nu.sum(1)   # Calculate delta by summing along rows
n_reactions = np.shape(nu)[0]   # Number of reactions = number of rows in nu
n_species = np.shape(nu)[1]     # Number of species = number of columns in nu

### Initial conditions
y0 = np.array([[0.5,0.5,0,0]]).T
P = 2.5  #atm
Pref = 1 #atm

### Guess reaction extents
xip_g = 0.1*np.ones(len(rxns)) # Guess 0.1 for all by default. Can add another line with specifics if desired

###
def Gmin(U):
    
    xip = np.reshape(U,[len(U),1]) 
    y = (y0+np.dot(nu.T,xip))/(1+np.dot(delta,xip))
    a = P/Pref*y
    
    ### First term
    G_term1 = 0 
    for i in range(n_reactions):
        G_term1 = G_term1-xip[i]*np.log(K[i,0])      

    ### Second term
    G_term2 = 0 
    for j in range(n_species):
        valj = y0[j]
        for i in range(n_reactions):
            valj = valj + nu[i,j]*xip[i]
        if a[j]>0:
            valj = valj * np.log(a[j])
        else:
            valj = 0
        G_term2 = G_term2 + valj

    Gm = G_term1 + G_term2
    
    return Gm

def ineq1(U):
    xip = U.reshape(-1,1)
    y = (y0+np.dot(nu.T,xip))/(1+np.dot(delta,xip))
    greaterthanzero = y.T[0].tolist()
    
    return greaterthanzero
    
### Run minimization
soln = minimize(Gmin,xip_g,method='trust-constr',constraints=dict(type='ineq',fun=ineq1))

### Report whether solver successful and solution
print('Success =',soln.success,'\n')

print('Gmin = \n',soln.fun,'\n')

xipsoln = np.reshape(soln.x,[len(soln.x),1]) 
print('xip = \n',xipsoln,'\n')

ysoln = (y0+np.dot(nu.T,xipsoln))/(1+np.dot(delta,xipsoln))
print('y = \n',ysoln)

Success = True 

Gmin = 
 -1.9434072584230624 

xip = 
 [[0.46639567]] 

y = 
 [[0.06297611]
 [0.06297611]
 [0.87404777]
 [0.        ]]


## Example 3: ethanol conversion

In [3]:
######## Gibbs energy minimization approach for solving chemical equilibrium

### Import packages

import numpy as np
from scipy.optimize import minimize

### Stoichiometric matrix
nu = np.array([[-1,1,1,0,0,0,0],
               [-2,0,1,1,0,0,0],
               [-1,0,0,0,1,1,0]])

### Reaction thermodynamics
dHrxn = np.array([[45.3,-24.4,68.5]]).T*1000  # J / mol rxn
dSrxn = np.array([[125.6,-35.5,98.2]]).T      # J / mol rxn
T = 473
R = 8.314
dGrxn = dHrxn-T*dSrxn
K = np.exp(-dGrxn/R/T)

### Initial conditions
y0 = np.array([[1,0,0,0,0,0,0]]).T
P = 2.5  #atm
Pref = 1 #atm

### Choose reactions
rxns = [0,1,2] # Specify a list (e.g., [0,2] will pick the first and third)

nu=nu[rxns,:]
K = K[rxns,:]

delta = nu.sum(1)   # Calculate delta by summing along rows
n_reactions = np.shape(nu)[0]   # Number of reactions = number of rows in nu
n_species = np.shape(nu)[1]     # Number of species = number of columns in nu

### Guess reaction extents
xip_g = 0.1*np.ones(len(rxns)) # Guess 0.1 for all by default. Can add another line with specifics if desired

###
def Gmin(U):
    
    xip = np.reshape(U,[len(U),1])   
    y = (y0+np.dot(nu.T,xip))/(1+np.dot(delta,xip))
    a = P/Pref*y
    
    ### First term
    G_term1 = 0 
    for i in range(n_reactions):
        G_term1 = G_term1-xip[i]*np.log(K[i,0])      

    ### Second term
    G_term2 = 0 
    for j in range(n_species):
        valj = y0[j]
        for i in range(n_reactions):
            valj = valj + nu[i,j]*xip[i]
        if a[j]>0:
            valj = valj * np.log(a[j])
        else:
            valj = 0
        G_term2 = G_term2 + valj

    Gm = G_term1 + G_term2
    
    return Gm

def ineq1(U):
    xip = xip = U.reshape(-1,1)
    y = (y0+np.dot(nu.T,xip))/(1+np.dot(delta,xip))
    greaterthanzero = y.T[0].tolist()
    
    return greaterthanzero
    
### Run minimization
soln = minimize(Gmin,xip_g,method='trust-constr',constraints=dict(type='ineq',fun=ineq1))

### Report whether solver successful and solution
print('Success =',soln.success,'\n')

print('Gmin = \n',soln.fun,'\n')

xipsoln = np.reshape(soln.x,[len(soln.x),1]) 
print('xip = \n',xipsoln,'\n')

ysoln = (y0+np.dot(nu.T,xipsoln))/(1+np.dot(delta,xipsoln))
print('y = \n',ysoln)

Success = True 

Gmin = 
 -3.201982654010804 

xip = 
 [[0.94393275]
 [0.00735936]
 [0.00955451]] 

y = 
 [[0.01627552]
 [0.48320394]
 [0.48697123]
 [0.00376729]
 [0.004891  ]
 [0.004891  ]
 [0.        ]]
